## Challenge exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Unlike in the challenge exercise for c_estimator.ipynb, assume that your measurements of r, h and V are all rounded off to the nearest 0.1. Simulate the necessary training dataset. This time, you will need a lot more data to get a good predictor.

Hint (highlight to see):
<p style='color:white'>
Create random values for r and h and compute V. Then, round off r, h and V (i.e., the volume is computed from the true value of r and h; it's only your measurement that is rounded off). Your dataset will consist of the round values of r, h and V. Do this for both the training and evaluation datasets.
</p>

Now modify the "noise" so that instead of just rounding off the value, there is up to a 10% error (uniformly distributed) in the measurement followed by rounding off.

In [1]:
import tensorflow as tf
import shutil
print(tf.__version__)

In [14]:
import pandas as pd
import numpy as np
from typing import Tuple, Dict

In [3]:
tf.enable_eager_execution()

# Creating fake data

In [4]:
import math

def add_round_noise(v: float) -> float:
    return math.floor(v * 10 + 0.5) / 10

def volume(r: float, h: float) -> float:
    return add_round_noise(math.pi * r ** 2 * h)

volume(2, 3)

37.7

In [5]:
def rand(data_size, min_val, max_val):
    r = np.random.random(data_size)
    return list((r + min_val) * (max_val - min_val))

rand(3, 0.5, 2.0)

[1.4017486986351138, 2.056240513402428, 0.7602382915174124]

In [18]:
def create_data(file_path, data_size):
    df = pd.DataFrame({
        'r': rand(train_data_size, 0.5, 2.0),
        'h': rand(train_data_size, 0.5, 2.0),
    })
    df['v'] = df.apply(lambda x: volume(x['r'], x['h']), axis=1)
    df.to_csv(file_path, index=False)

In [19]:
train_data_size = 100000
train_csv_path = 'cylinder_train.csv'
eval_data_size = 1000
eval_csv_path = 'cylinder_eval.csv'

create_data(train_csv_path, train_data_size)
create_data(eval_csv_path, eval_data_size)

In [8]:
train_csv_path = 'cylinder.csv'
df.to_csv(csv_path, index=False)

In [20]:
!head 'cylinder_train.csv'

h,r,v
1.8747613541793857,1.9267289719904372,21.9
1.24569591677789,0.9947033772202674,3.9
1.4073173078475179,1.7261079821653103,13.2
1.8465481758331626,1.1076681520444904,7.1
1.971534514923845,1.1481340571573242,8.2
1.5894266511156692,1.1241953023860904,6.3
1.4981473304021988,1.8393553696653993,15.9
1.2655170400798577,1.7776506751828327,12.6
1.2216866111472562,1.123904636760331,4.8


In [21]:
!head 'cylinder_eval.csv'

h,r,v
1.3096461867317868,2.1156277491964888,18.4
1.8080293291494067,1.8271944309932135,19.0
1.6884540142949689,1.3172170103851422,9.2
1.982964837668804,1.2819480580855733,10.2
1.3278072932707254,1.6239554577430941,11.0
1.5637943818549793,1.2442584225365176,7.6
1.0400917854019234,0.8440819248781215,2.3
0.9558163996680948,1.439910571094971,6.2
1.483181526370225,2.112109982336121,20.8


# Data Function

In [17]:
FEATURE_NAMES = ['h', 'r']

def parse_row(row: str) -> Tuple[Dict[str, tf.Tensor], tf.Tensor]:
    cols = tf.decode_csv(row, record_defaults=[0.0, 0.0, 0.0])
    label = cols.pop()
    features = dict(zip(FEATURE_NAMES, cols))
    return features, label

parse_row('1.0, 2.0, 3.0')

({'h': <tf.Tensor: id=15, shape=(), dtype=float32, numpy=1.0>,
  'r': <tf.Tensor: id=16, shape=(), dtype=float32, numpy=2.0>},
 <tf.Tensor: id=17, shape=(), dtype=float32, numpy=3.0>)

In [28]:
def training_input_fn(path, batch_size = 128):
    dataset = tf.data.TextLineDataset(path).skip(1).map(parse_row).cache()
    return dataset.repeat().shuffle(1024).batch(batch_size)

def eval_input_fn(path, batch_size = 128):
    return tf.data.TextLineDataset(path).skip(1).map(parse_row).batch(batch_size)

In [29]:
next(iter(training_input_fn(train_csv_path, batch_size=2)))

({'h': <tf.Tensor: id=185, shape=(2,), dtype=float32, numpy=array([1.2546396, 2.0149477], dtype=float32)>,
  'r': <tf.Tensor: id=186, shape=(2,), dtype=float32, numpy=array([1.0351797, 1.1138978], dtype=float32)>},
 <tf.Tensor: id=187, shape=(2,), dtype=float32, numpy=array([4.2, 7.9], dtype=float32)>)

In [30]:
next(iter(eval_input_fn(eval_csv_path, batch_size=2)))

({'h': <tf.Tensor: id=223, shape=(2,), dtype=float32, numpy=array([1.3096461, 1.8080293], dtype=float32)>,
  'r': <tf.Tensor: id=224, shape=(2,), dtype=float32, numpy=array([2.1156278, 1.8271945], dtype=float32)>},
 <tf.Tensor: id=225, shape=(2,), dtype=float32, numpy=array([18.4, 19. ], dtype=float32)>)

# Model

In [33]:
feature_columns = [tf.feature_column.numeric_column(name) for name in FEATURE_NAMES]
model_dir = 'cy_model'
lr = 0.001

shutil.rmtree(model_dir, ignore_errors=True)

model = tf.estimator.DNNRegressor(
    hidden_units=[128, 128],
    feature_columns=feature_columns,
    model_dir=model_dir,
    optimizer=tf.train.AdamOptimizer(lr),
    config=tf.estimator.RunConfig(tf_random_seed=1),
)

INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_device_fn': None, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_service': None, '_task_type': 'worker', '_log_step_count_steps': 100, '_train_distribute': None, '_is_chief': True, '_experimental_max_worker_delay_secs': None, '_model_dir': 'cy_model', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_task_id': 0, '_protocol': None, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_global_id_in_cluster': 0, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_experimental_distribute': None, '_eval_distribute': None, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f901be1d0>, '_session_creation_timeout_secs': 7200}


In [36]:
model.train(lambda: training_input_fn(train_csv_path, batch_size=128), steps=1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into cy_model/model.ckpt.


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(128, 2), b.shape=(2, 128), m=128, n=128, k=2
	 [[node dnn/hiddenlayer_0/MatMul (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]
  (1) Internal: Blas GEMM launch failed : a.shape=(128, 2), b.shape=(2, 128), m=128, n=128, k=2
	 [[node dnn/hiddenlayer_0/MatMul (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[dnn/head/gradients/dnn/hiddenlayer_0/MatMul_grad/MatMul_1/_91]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'dnn/hiddenlayer_0/MatMul':
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib